In [103]:
#DTM implementaition
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from nltk.corpus import stopwords
from nltk.corpus import opinion_lexicon
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
#list all of the mda_files
MDA_FILES = os.listdir("data/mda")
MDA_FILES = [file for file in MDA_FILES if file!=".DS SOTRE"]
positive_lexicon = opinion_lexicon.positive()

In [105]:
#clean text function implemantation
stopwords_lex = set(stopwords.words('english'))
def clean_text(mda_text):
    mda_words = mda_text.split()
    mda_words_cleaned_symbols =[words for words in mda_words if words.isalpha()]
    mda_words_harmonized = [words.lower() for words in mda_words_cleaned_symbols]
    mda_words_cleaned = [words for words in mda_words_harmonized if words not in stopwords_lex]
    return mda_words_cleaned

In [106]:
# Extract text from all MD&A filings (full sample)
mda_text = {}
#designig a dic for better view
for file in MDA_FILES:
    cik = file.split('_')[0]
    filing_date = file.split('_')[2].split('.')[0]
    cik_filing_date = cik + '_' + filing_date
    with open('data/mda/' + file) as f:
        text = f.read()
    mda_text[cik_filing_date]=text

In [107]:
df_text = pd.DataFrame(mda_text, index=['text']).T
df_text.reset_index(inplace=True)
df_text.columns = ['cik_filing_date', 'mda_text']

In [108]:
df_text['cik'] = df_text['cik_filing_date'].apply(lambda x:x.split('_')[0])
#type(df_text['cik'][0])  --> output str
#parse cik data from str to numeric
df_text['cik'] = pd.to_numeric(df_text['cik'])

In [109]:
df_text['filing_date']=df_text['cik_filing_date'].apply(lambda x:x.split('_')[1])
df_text['filing_date']=pd.to_datetime(df_text['filing_date'] , format='%Y-%m-%d')

In [110]:
#final arrangment of datafram
df_text.drop(columns=['cik_filing_date'] , inplace=True)
df_text = df_text[['cik', 'filing_date', 'mda_text']]

In [111]:
df_text['mda_text_clean'] = df_text['mda_text'].apply(clean_text)
df_text['num_cleaned_words'] = df_text['mda_text_clean'].apply(lambda x: len(x))
df_text['mda_text_clean_str'] = df_text['mda_text_clean'].apply(lambda x: " ".join(x))

In [113]:
#dtm vector #codes below are duplicated from a specific site
count_vec = CountVectorizer(vocabulary=positive_lexicon)
dtm_pos_words = count_vec.fit_transform(df_text['mda_text_clean_str'])
dtm_pos_words.toarray()
df_dtm_pos_words = pd.DataFrame(dtm_pos_words.toarray())
df_dtm_pos_words.columns = count_vec.vocabulary_.keys()

In [119]:
df_text['positive count']=df_dtm_pos_words.sum(axis=1)
df_text=df_text[df_text['positive count']>=100]
df_text['phi_pos'] = (df_text['positive count']/df_text['num_cleaned_words'])
df_te

<ipython-input-119-b97f1b1bc14f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['positive count']=df_dtm_pos_words.sum(axis=1)
